<a href="https://colab.research.google.com/github/erberry/ThinkML/blob/main/train_bloom.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 在新数据集上对 bloom-560m 模型进行微调

# 安装依赖

In [ ]:
!pip install transformers==4.29.0
!pip install git+https://github.com/huggingface/accelerate
!pip install datasets
!pip install torch
!pip install pandas

查看 GPU 信息：

显卡配置：A100 40GB

In [ ]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

Thu Jun 15 13:15:41 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.85.12    Driver Version: 525.85.12    CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA A100-SXM...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   33C    P0    42W / 400W |      0MiB / 40960MiB |      0%      Default |
|                               |                      |             Disabled |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

# 训练 tokenizer （这一步可跳过，后续的模型训练可以使用原 tokenizer）

这里使用新数据集训练 tokenizer 的本意是担心原模型的 tokenizer 不认识新数据集中的词汇。

经过几次试验，发现原模型的词汇量挺大的，没必要再训练。

加载数据集

In [ ]:
from datasets import load_dataset
law_dataset = load_dataset("erberry/github-lll")

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split:   0%|          | 0/156824 [00:00<?, ? examples/s]

Dataset parquet downloaded and prepared to /root/.cache/huggingface/datasets/erberry___parquet/erberry--github-lll-c691dbf5ef3b534e/0.0.0/14a00e99c0d15a23649d0db8944380ac81082d4b021f398733dd84f3a6c569a7. Subsequent calls will reuse this data.


  0%|          | 0/1 [00:00<?, ?it/s]

In [ ]:
print(law_dataset)

DatasetDict({
    train: Dataset({
        features: ['Unnamed: 0', 'fname', 'text'],
        num_rows: 156824
    })
})


加载原 tokenizer

In [ ]:
from transformers import AutoTokenizer

old_tokenizer = AutoTokenizer.from_pretrained("bigscience/bloom-560m")

训练 tokenizer

每1000条数据一个批次，分批训练。

训练目标是达到 5w 词汇量，别问我为什么，随便定的。

漫长、无聊的训练开始，他怎么就这么慢！

如果只是为了体验一下训练过程，可以只取一部分dataset，例如这样：



```
import torch

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

small_dataset = law_dataset["train"][:3000] #取前3000条

training_corpus = (
    law_dataset[i : i + 1000]["text"]
    for i in range(0, len(law_dataset), 1000)
)

tokenizer = old_tokenizer.train_new_from_iterator(training_corpus, 50000, device=device)
tokenizer.save_pretrained("chinese-bloom-law")
```



经过约40分钟，训练完成

In [ ]:
import torch

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

training_corpus = (
    law_dataset["train"][i : i + 1000]["text"]
    for i in range(0, len(law_dataset["train"]), 1000)
)

tokenizer = old_tokenizer.train_new_from_iterator(training_corpus, 50000, device=device)
tokenizer.save_pretrained("chinese-bloom-law")

('chinese-bloom-law/tokenizer_config.json',
 'chinese-bloom-law/special_tokens_map.json',
 'chinese-bloom-law/tokenizer.json')

In [ ]:
授权 colab 访问 huggingface，在弹出的页面中输入自己在 huggingface 的 access token （在个人主页的设置里，需要拥有写权限的token）
from huggingface_hub import notebook_login

notebook_login()

In [ ]:
tokenizer.push_to_hub("chinese-bloom-law")

CommitInfo(commit_url='https://huggingface.co/erberry/chinese-bloom-law/commit/3d35975ade3de9c27bf1412b4df04122609baa90', commit_message='Upload tokenizer', commit_description='', oid='3d35975ade3de9c27bf1412b4df04122609baa90', pr_url=None, pr_revision=None, pr_num=None)

验证

In [ ]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("erberry/chinese-bloom-law")
encoding = tokenizer.encode("人民检察院建议人民法院适用速裁程序的案件，起诉书内容可以适当简化，重点写明指控的事实和适用的法律")
print(encoding)
tokenizer.decode(encoding)

[1084, 1563, 525, 33596, 2935, 170, 12307, 1212, 10078, 7723, 170, 1426, 6532, 12359, 6904, 212, 27713]


'人民检察院建议人民法院适用速裁程序的案件，起诉书内容可以适当简化，重点写明指控的事实和适用的法律'

# 微调模型

### 加载原模型

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM

old_model = AutoModelForCausalLM.from_pretrained("bigscience/bloom-560m")

tokenizer = AutoTokenizer.from_pretrained("bigscience/bloom-560m")

### 加载数据集

In [ ]:
from datasets import load_dataset
law_dataset = load_dataset("erberry/github-lll")

将数据集拆分为训练集合和测试集合

训练集合用来对模型进行训练，测试集合用来验证模型在新数据集上的表现（判断是否过拟合）

In [ ]:
print(law_dataset)
dataset = law_dataset['train'].train_test_split(test_size=0.2)
print(dataset)
print(dataset["train"][:8])
print(dataset["test"][:8])

DatasetDict({
    train: Dataset({
        features: ['Unnamed: 0', 'fname', 'text'],
        num_rows: 156824
    })
})
DatasetDict({
    train: Dataset({
        features: ['Unnamed: 0', 'fname', 'text'],
        num_rows: 125459
    })
    test: Dataset({
        features: ['Unnamed: 0', 'fname', 'text'],
        num_rows: 31365
    })
})
{'Unnamed: 0': [99147, 9624, 68655, 32735, 144823, 23890, 57218, 153426], 'fname': ['广东省湿地保护条例(2022 11 30).md.txt', '山东省劳动合同条例(2013 08 01).md.txt', '浙江省计量监督管理条例.md.txt', '郑州市城市房地产市场管理条例(2010 12 20).md.txt', '河南省警务辅助人员条例(2020 09 27).md.txt', '中国（上海）自由贸易试验区条例(2014 07 25).md.txt', '长江三峡工程建设移民条例(2011 01 08).md.txt', '证券投资基金法(2015 04 24).md.txt'], 'text': ['广东省湿地保护条例(2022 11 30).md.txt。第四条 县级以上人民政府应当建立湿地保护工作协调机制，组织协调、研究解决湿地保护工作中的重大问题。各级人民政府对本行政区域内湿地保护负总责，加强湿地保护工作，将湿地保护纳入国民经济和社会发展规划，保障湿地保护、修复和补偿的资金投入。沿海县级以上人民政府应当加强对红树林湿地的保护，采取措施保护和恢复红树林，解决红树林湿地保护工作中的重大问题。', '山东省劳动合同条例(2013 08 01).md.txt。第十条 用人单位有权了解劳动者与劳动合同直接相关的健康状况、工作经历、知识技能等基本情况，核对劳动者的居民身份证、居住证等相关证件，劳动

想了解关于数据集的选择和生成，见[准备数据集](https://mp.weixin.qq.com/s?__biz=Mzk0MDI2Nzc3Mw==&mid=2247484525&idx=1&sn=b46800ad10ac3145e77a3febcb664bd3&chksm=c2e505fff5928ce909cdd778acfb91f2bb17e814efcf1875b4628c0352d17685d6d7454c9776#rd)

### 进行 tokenization

将数据集中的文本进行 tokenization ，转为算法可识别的数值类型：token

通过 dataset.map 方法分批进行 tokenization，有如下优点：

- 数据不会被一次性加载进内存，而是分批加载
- tokenization 结果写入 dataset，进行持久化

In [ ]:
def preprocess_function(examples):
    return tokenizer(examples["text"])

In [ ]:
dataset_token = dataset.map(
    preprocess_function,
    batched=True,
    remove_columns=dataset["train"].column_names,
)

由于模型计算需要输入的数据长度是一致的。

使用 DataCollator 将同一批句子的 token 的长度填充至一致。

将填充的工作从 tokenization 转移到了训练阶段，有如下优点：

- 只需要同一批句子的 token 长度一致（如果在 tokenization 阶段填充，需要将所有句子填充至整个数据集最长句子的长度）

In [ ]:
from transformers import DataCollatorForLanguageModeling

tokenizer.pad_token = tokenizer.eos_token
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)

想了解更多关于自然语言处理的 tokenization，见[NLP 之 Tokenization](https://mp.weixin.qq.com/s?__biz=Mzk0MDI2Nzc3Mw==&mid=2247484512&idx=1&sn=7895bd5a3a2c2931bd4312cccf3fb0fc&chksm=c2e505f2f5928ce49d30ef0022a5231945ed91dd61b2408f09d2dbe763b221cb4911e4a570b7#rd)

### 开始训练

开始漫长的训练过程。

一张显存 40GB 的 A100 显卡，训练时间大约3个小时。

对于显存或者内存不足的情况，可以调节三个参数：

- 设置半精度： fp16=True，同时会降低模型的准确度
- 降低批量处理： per_device_train_batch_size 和 per_device_eval_batch_size，同时会加长训练时间

对于缩短训练时间，可以调节四个参数：

- num_train_epochs: 训练的 Epoch 数量，较小的值可以缩短训练时间。但会出现由于训练不足，模型准确度低。
- 增加批量处理：调高 per_device_train_batch_size 和 per_device_eval_batch_size，但同时会增加内存使用
- learning_rate: 学习率，较大的值可以使模型更快地收敛，但也可能导致模型不稳定或无法收敛。
- max_steps: 最大训练步数，可以理解训练是一个循环，每个循环从数据集中取一批数据对模型进行更新，max_steps 代表了循环的次数。max_steps 设置后，将不会使用 epoch方式。

In [ ]:
from transformers import TrainingArguments, Trainer

training_args = TrainingArguments(
    output_dir="chinese-bloom-law-run",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    weight_decay=0.01,
    fp16=True, # 设置半精度，降低显存要求
    per_device_train_batch_size=8, # 指定一次向设备传输几批数据，决定了使用的显存大小
    per_device_eval_batch_size=8,
)

trainer = Trainer(
    model=old_model,
    args=training_args,
    train_dataset=dataset_token["train"],
    eval_dataset=dataset_token["test"],
    data_collator=data_collator,
)

trainer.train()

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:407: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
You're using a BloomTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Epoch,Training Loss,Validation Loss
1,1.518400,1.518537
2,1.182200,1.399636
3,0.821200,1.457843


TrainOutput(global_step=47049, training_loss=1.2659977867072791, metrics={'train_runtime': 9651.7759, 'train_samples_per_second': 38.996, 'train_steps_per_second': 4.875, 'total_flos': 1.152351182629847e+17, 'train_loss': 1.2659977867072791, 'epoch': 3.0})

经过2小时40分钟，完成了3轮 epoch。

每个 epoch 会使用数据集中的所有数据进行训练，训练完成后，再在测试数据上进行一次验证。

所以上面的表格中有三行数据，每行代表了在整个数据集上训练完成的**训练损失和验证损失**。

第三次 epoch，出现了训练损失降低，而验证损失升高，代表出现了**过拟合现象**。

过拟合意味着模型在它见过的数据上表现好，但在没有见过的数据上表现差。

epoch 轮数过多，模型对训练数据出现了刻板印象，容易导致过拟合；而轮数过少，则容易出现欠拟合。

### 进行模型评估

计算模型困惑度，关注模型在测试数据集上的预测准确性，越低越好

In [ ]:
import math

eval_results = trainer.evaluate()
print(f"Perplexity: {math.exp(eval_results['eval_loss']):.2f}")

Perplexity: 4.30


从 Perplexity 指标看，训练结果还不错。原模型的 Perplexity 是 8.9。

### 保存模型

把训练好的模型保存到 huggingface。需要登录 huggingface。


In [ ]:
# 授权 colab 访问 huggingface，在弹出的页面中输入自己在 huggingface 的 access token （在个人主页的设置里，需要拥有写权限的token）
from huggingface_hub import notebook_login

notebook_login()

In [ ]:
trainer.save_model("chinese-bloom-law-beta")
beta_model = AutoModelForCausalLM.from_pretrained("./chinese-bloom-law-beta")

In [ ]:
beta_model.push_to_hub("chinese-bloom-law-beta")

Upload 1 LFS files:   0%|          | 0/1 [00:00<?, ?it/s]

pytorch_model.bin:   0%|          | 0.00/2.24G [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/erberry/chinese-bloom-law-beta/commit/ea6141d7b7dbf127efb63155e143b4321eebb8f5', commit_message='Upload BloomForCausalLM', commit_description='', oid='ea6141d7b7dbf127efb63155e143b4321eebb8f5', pr_url=None, pr_revision=None, pr_num=None)

In [ ]:
inputs = tokenizer("防卫过当", return_tensors="pt").input_ids
outputs = beta_model.generate(inputs, max_new_tokens=200, do_sample=True, top_k=50, top_p=0.95)
result = tokenizer.batch_decode(outputs, skip_special_tokens=True)
print(result)

['防卫过当法(2021 06 10).md.txt。第十五条 被请求协助的防卫行为应当有利于维护国家主权、统一、领土完整和安全，不违背中国主权的原则，符合中国法律和有关国际条约的规定，不损害中国主权、安全和利益的。有关军事机关可以按照约定，请求被请求协助的防卫行为实施单位作为一方履行协助义务。请求书应当用中文制作，附存在协助义务的履行函，以及相关文书译制格式。国防科技工业、商务等部门或者军队其他有关部门按照约定提供协助的义务，在被请求人提出请求时视为被请求人履行协助义务。有关军事机关认为履行协助义务需要的，可以根据其能力、成效等直接向其下达协助请求，被请求人应当按照要求予以配合。被请求协助的防卫行为不违背中国国家主权、安全和利益的，可以不要求其提供协助。国防科技工业、商务等部门或者军队其他有关部门不履行协助义务的，由被请求人提出意见']


In [ ]:
law_model = AutoModelForCausalLM.from_pretrained("erberry/chinese-bloom-law-beta")

In [ ]:
inputs = tokenizer("劳动合同", return_tensors="pt").input_ids
outputs = beta_model.generate(inputs, max_new_tokens=200, do_sample=True, top_k=50, top_p=0.95)
result = tokenizer.batch_decode(outputs, skip_special_tokens=True)
print(result)

['劳动合同法(2012 12 28).md.txt。第九十二条 本法自2004年1月1日起施行。1988年9月18日国务院发布的《中华人民共和国劳动合同法》同时废止。附录。劳动法律监督规定。总则。劳动合同的基本形式。劳动法律监督规定的劳动法律监督。法人。有限责任公司的劳动合同。有限合伙企业的劳动合同。外商投资企业的劳动合同。集体合同。劳动合同的起算时间。劳动合同的履行。劳动合同的期限。劳动合同的履行率。用人单位与劳动者约定的劳动合同期限， salat是劳动合同的履行时间，也是定员时的标准。用人单位与劳动者商定的合同期限与实际履行的时间不一致时，以实际履行的时间为准。用人单位与劳动者约定的劳动合同期限短于法定的或者国家规定的劳动合同期限， salat是劳动合同的履行时间。用人单位未按照法律规定和集体合同约定履行劳动合同，劳动者要求解除或者终止劳动合同，用人单位没有非法理由的，视为用人单位与劳动者约定的劳动合同履行期限短于法定的或者国家规定的期限']
